# High-Level Setup & Imports

In [1]:
# General Python Packages
import os, time, numbers, math

# Torch Packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import lr_scheduler, SGD
from torch.autograd import Variable
from torch import nn
from torch.nn import DataParallel
from torch.nn import Module

# General Analytics Packages
import pandas as pd
import numpy as np

# Visualization / Image Packages
import matplotlib.pyplot as plt
from PIL import Image

# Randomization Functions
from random import random as randuni

In [2]:
# Put MatPlotLib in interactive mode
plt.ion()

# Plot graphics inline in the notebook
%matplotlib inline

# Define Classes and Functions

### Image Data Utility Classes

In [3]:
def is_image_file(fname):
    """Checks if a file is an image.
    Args:
        fname (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return fname.lower().endswith('.png')

def create_label_maps(details_df):
    """ Take a descriptive dataframe and extract the unique labels and map to index values
    Args:
        details_df: Dataframe with the image details
    Returns:
        label_list: list of unique labels in the dataframe
        label_to_index: map from labels to indices
    """
    """ TODO: Research paper also excludes these labels but need to figure out how to handle
              cases that have these as positive findings (completely exclude?)
    excluded_labels = ['Edema','Hernia','Emphysema','Fibrosis','No Finding'
                      'Pleural_Thickening','Consolidation']
    """
    excluded_labels = ['No Finding']
    
    label_groups = details_df['Finding Labels'].unique()
    unique_labels = set([label for sublist in label_groups.tolist() for label in sublist.split('|')])
    
    # Drop some label that we do not want to include
    unique_labels = [l for l in unique_labels if l not in excluded_labels]

    index_to_label = {idx: val for idx, val in enumerate(unique_labels)}
    label_to_index = {val: idx for idx, val in index_to_label.items()}

    label_list = list(label_to_index.keys())

    return label_list, label_to_index

def create_image_list(dir):
    """ Create a full list of images available 
    Args:
        dir (string): root directory of images with subdirectories underneath
                      that have the .png images within them
    Returns:
        image_list: list of tuples with (image_name, full_image_path)
    """
    image_list = []
    dir = os.path.expanduser(dir)
    for subfolder in sorted(os.listdir(dir)):
        d = os.path.join(dir, subfolder)
        if not os.path.isdir(d):
            continue
        for subfolder_path, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(subfolder_path, fname)
                    image_list.append((fname, path))
    return image_list

def pil_loader(path):
    """ Opens path as file with Pillow (https://github.com/python-pillow/Pillow/issues/835)
    Args:
        path (string): File path to the image
    Returns:
        img: Image in RGB format
    """
    f = open(path, 'rb')
    return Image.open(f)
    #with open(path, 'rb') as f:
    #    return Image.open(f)
        #with Image.open(f) as img:
        #    return img.load()
        
def imshow(inp, title=None):
    """ Convert tensor array to an image (only use post-dataset transform) """
    inp = inp[0]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

### Torch Dataset Definition

In [4]:
class XrayImageSet(Dataset):
    """
    Args:
        image_root (string): root directory of the images in form image/subfolder/*.png
        csv_file (string): path to the CSV data file
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
     Attributes:
        labels (list): list of the possible label names.
        label_to_index (dict): look from label name to a label index
        imgs (list): List of (filename, image path) tuples
    """
    
    def __init__(self, image_root, csv_file, transform=None, target_transform=None, loader = pil_loader):
        """ Create an instance of the Xray Dataset """
        img_details = pd.read_csv(csv_file)
        
        labels, label_to_index = create_label_maps(img_details)
        imgs = create_image_list(image_root)

        self.imgs = imgs
        self.image_details = img_details
        self.image_root = image_root
        self.labels = labels
        self.label_to_index = label_to_index
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.max_label_index = max(label_to_index.values())

    def __getitem__(self, index):
        """ Get image,labels pair by index
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        fname, path = self.imgs[index]
        target = self.get_one_hot_labels(fname)
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """ Calculate length of the dataset (number of images) """
        return len(self.imgs)
    
    def get_labels(self, fname):
        """ Return the label string for the file """
        return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]
    
    def one_hot_labels(self, labels):
        """ Convert the labels string (with each label separated by |) into 1-hot encoding """
        if labels == None:
            return None
        
        split_label_indices = [self.label_to_index.get(label)
                               for label in labels.split('|')
                               if label != 'No Finding']
        
        out = [1 if idx in split_label_indices else 0 for idx in range(self.max_label_index+1)]
        # This code UNHOTs the labels:
        # out = '|'.join([index_to_label.get(idx) for idx, val in enumerate(one_hot_tuple) if val == 1])
        return out

    def get_one_hot_labels(self, fname):
        """ Get the 1-hot encoded label array for the provided file """
        labels = self.get_labels(fname)
        one_hot_labels = self.one_hot_labels(labels)
        return torch.FloatTensor(one_hot_labels)

### Model Training Output Writer

In [5]:
class printer_writer:
    def __init__(self, output_folder_path):
        self.start_time = time.strftime('%Y%m%d-%Hh%Mm%Ss')
        
        self.outprefix = output_folder_path + '/' + self.start_time
        
        # Print Output File
        self.print_out_path = self.outprefix + '_print.txt'
        self.print_out_file = open(self.print_out_path, 'w', 1)
        
    def printw(self, string):
        print(string)
        try:
            self.print_out_file.write(string + "\n")
        except: # Ignore errors
            pass
        
    def save_checkpoint(self, epoch, model, optimizer, scheduler, val_error):
        model_out_path = self.outprefix + '_model_' + str(epoch+1) + '.tar'
        
        torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
        
    def close(self):
        self.print_out_file.close()

### Model Training Procedure

In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, outfolder = '/user/xrayproj/output/'):
    since = time.time()
    scribe = printer_writer(outfolder)

    for epoch in range(num_epochs):
        scribe.printw('Epoch {}/{}'.format(epoch, num_epochs - 1))
        scribe.printw('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0
            obs_counter = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # Store statistics (convert from autograd.Variable to float/int)
                loss_val = loss.data[0]
                correct_val = torch.sum( ((outputs.sigmoid()>0.5) == (labels>0.5)).long() ).data[0]
                
                running_loss += loss_val
                running_corrects += correct_val
                
                obs_counter += len(inputs)
                
                batch_loss = 1.0 * loss_val / len(inputs)
                batch_acc = 1.0 * correct_val / len(inputs)
                status = ' |~~ {}@{}  Loss: {:.6f} Acc: {:.4f}'.format(
                    phase, obs_counter, batch_loss, batch_acc)
                scribe.printw(status)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            scribe.printw('{}  Loss: {:.6f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # Store the model on disk
            if phase == 'val':
                scheduler.step(epoch_loss)
                if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                    scribe.save_checkpoint(epoch, model, optimizer, None, epoch_loss)
                else:
                    scribe.save_checkpoint(epoch, model, optimizer, scheduler, epoch_loss)

    time_elapsed = time.time() - since
    scribe.printw('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    scribe.close()

    return model

### Customized Binary Cross Entropy Loss Function

In [7]:
class BCEWithLogitsImbalanceWeightedLoss(Module):
    def __init__(self, class_weight=None, size_average=True):
        super(BCEWithLogitsImbalanceWeightedLoss, self).__init__()
        self.size_average = size_average

    def forward(self, input, target):
        return self.imbalance_weighted_bce_with_logit(input, target, size_average=self.size_average)
    
    def imbalance_weighted_bce_with_logit(self, input, target, size_average=True):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

        # Determine |P| and |N|
        positive_labels = target.sum()
        negative_labels = (1-target).sum()

        # Upweight the less common class (very often the 1s)
        beta_p = (positive_labels + negative_labels) / positive_labels
        beta_n = (positive_labels + negative_labels) / negative_labels

        # Adjust the losses accordingly
        loss_weight = target * beta_p + (1-target) * beta_n

        loss = loss * loss_weight

        if size_average:
            return loss.mean()
        else:
            return loss.sum()

### Define model types

In [8]:
def ResNetBase(base_size = 18, only_update_fc = True, pretrain = True):
    """ ResNet 18 with only final FC layer updatable """
    m = None
    if base_size == 18:
        m = models.resnet18(pretrained=pretrain)
    elif base_size == 34:
        m = models.resnet34(pretrained=pretrain)
    elif base_size == 50:
        m = models.resnet50(pretrained=pretrain)
    elif base_size == 101:
        m = models.resnet101(pretrained=pretrain)
    elif base_size == 152:
        m = models.resnet152(pretrained=pretrain)
    
    if only_update_fc:
        for param in m.parameters():
            param.requires_grad = False

    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

#### mean ± std. dev. of 7 runs, 10000000 loops each

#### Time for __get_item__
```
%timeit img_data_train[3] # 30.8 ms ± 544 µs per loop
```

#### Breakdown for __get_item__
```
%timeit img_data_train.imgs[8] # 63 ns ± 0.0057 ns per loop
%timeit img_data_train.get_one_hot_labels('00011558_012.png') # 8.72 ms ± 9.44 µs per loop
%timeit img_data_train.loader('/user/images/images_006/00011558_012.png') # 14.1 ms ± 3.41 µs per loop
%timeit img_data_train.transform(img) # 3.17 ms ± 1.32 µs per loop
```

#### Breakdown for loader() from __get_item__
```
%timeit open('/user/images/images_006/00011558_012.png', 'rb') # 7.72 µs ± 13.4 ns per loop
%timeit Image.open(f) # 37.5 µs ± 2.25 µs per loop
%timeit img.convert('RGB') # 498 µs ± 149 ns per loop
```

# Setup and Begin Training Model

### Data Setup

In [9]:
nn_input_size = 224 #1024
batch_size = 64
pin_mem_setting = True
num_gpus = torch.cuda.device_count()
num_workers = 10

print("Number of GPU: {}".format(num_gpus))

Number of GPU: 1


In [10]:
img_transforms_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [11]:
img_transforms_nontrain = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [12]:
img_data_train = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_train,
                              target_transform = None)

img_data_train.imgs = [img for i, img in enumerate(img_data_train.imgs) if i % 10 >= 3]

In [27]:
img_data_val   = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_nontrain,
                              target_transform = None)

img_data_val.imgs = [img for i, img in enumerate(img_data_val.imgs) if i % 10 in (1,2)]

In [28]:
img_data_test  = XrayImageSet(image_root = '/user/images_processed/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms_nontrain,
                              target_transform = None)

img_data_test.imgs = [img for i, img in enumerate(img_data_test.imgs) if i % 10 == 0]

In [29]:
train_set = set(img_data_train.imgs)
val_set = set(img_data_val.imgs)
test_set = set(img_data_test.imgs)
assert len(train_set.intersection(val_set)) == 0
assert len(train_set.intersection(test_set)) == 0
assert len(val_set.intersection(test_set)) == 0

In [30]:
print("Training Set Size: {}".format(len(img_data_train)))
print("Validation Set Size: {}".format(len(img_data_val)))
print("Test Set Size: {}".format(len(img_data_test)))

Training Set Size: 78484
Validation Set Size: 22424
Test Set Size: 11212


In [31]:
img_loader_train = DataLoader(img_data_train,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = num_workers,
                              pin_memory = pin_mem_setting)

img_loader_val   = DataLoader(img_data_val,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = num_workers,
                              pin_memory = pin_mem_setting)

In [32]:
dataloaders = {
    'train': img_loader_train,
    'val': img_loader_val
}

In [33]:
model_base = ResNetBase(base_size = 50, only_update_fc = False, pretrain = True)

In [34]:
model_ft = DataParallel(model_base).cuda()

### Setup learning rates and procedures

In [35]:
#criterion = BCEWithLogitsImbalanceWeightedLoss()
criterion_base = nn.BCEWithLogitsLoss()

#optimizer_ft = SGD(model_ft.module.fc.parameters(), lr=0.01, momentum=0.9)
optimizer_ft = SGD(model_ft.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
#lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
learning_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, patience=0)

In [36]:
criterion = criterion_base.cuda()

### Future code for allowing optimization of the base layer with a lower learning rate

```
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())

optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': opt.lr}
        ], lr=opt.lr*0.1, momentum=0.9)
```

# Begin Training Network

In [37]:
train_model(model_ft,
            criterion,
            optimizer_ft,
            learning_scheduler,
            num_epochs=10)

Epoch 0/9
----------
 |~~ train@32  Loss: 0.022544 Acc: 6.4062
 |~~ train@64  Loss: 0.020611 Acc: 8.5000
 |~~ train@96  Loss: 0.017324 Acc: 12.2188
 |~~ train@128  Loss: 0.013438 Acc: 13.4062
 |~~ train@160  Loss: 0.010819 Acc: 13.1875
 |~~ train@192  Loss: 0.008548 Acc: 13.2500
 |~~ train@224  Loss: 0.006244 Acc: 13.4688
 |~~ train@256  Loss: 0.005528 Acc: 13.4688
 |~~ train@288  Loss: 0.004888 Acc: 13.4375
 |~~ train@320  Loss: 0.007980 Acc: 12.9688
 |~~ train@352  Loss: 0.005736 Acc: 13.3438
 |~~ train@384  Loss: 0.007190 Acc: 13.1875
 |~~ train@416  Loss: 0.008099 Acc: 13.0625
 |~~ train@448  Loss: 0.007553 Acc: 13.1875
 |~~ train@480  Loss: 0.006185 Acc: 13.2812
 |~~ train@512  Loss: 0.005551 Acc: 13.3438
 |~~ train@544  Loss: 0.005663 Acc: 13.3750
 |~~ train@576  Loss: 0.005907 Acc: 13.3750
 |~~ train@608  Loss: 0.007327 Acc: 13.1875
 |~~ train@640  Loss: 0.008012 Acc: 13.2188
 |~~ train@672  Loss: 0.006433 Acc: 13.2812
 |~~ train@704  Loss: 0.008079 Acc: 13.1250
 |~~ train@736  

 |~~ train@5920  Loss: 0.004748 Acc: 13.2812
 |~~ train@5952  Loss: 0.006863 Acc: 13.0938
 |~~ train@5984  Loss: 0.006319 Acc: 13.0625
 |~~ train@6016  Loss: 0.004683 Acc: 13.3750
 |~~ train@6048  Loss: 0.004675 Acc: 13.3438
 |~~ train@6080  Loss: 0.003709 Acc: 13.4688
 |~~ train@6112  Loss: 0.005312 Acc: 13.4375
 |~~ train@6144  Loss: 0.004318 Acc: 13.3750
 |~~ train@6176  Loss: 0.003136 Acc: 13.6250
 |~~ train@6208  Loss: 0.005983 Acc: 13.1250
 |~~ train@6240  Loss: 0.006692 Acc: 13.0625
 |~~ train@6272  Loss: 0.006500 Acc: 13.0625
 |~~ train@6304  Loss: 0.004797 Acc: 13.4375
 |~~ train@6336  Loss: 0.005383 Acc: 13.3125
 |~~ train@6368  Loss: 0.004560 Acc: 13.4688
 |~~ train@6400  Loss: 0.006342 Acc: 13.0312
 |~~ train@6432  Loss: 0.005478 Acc: 13.1875
 |~~ train@6464  Loss: 0.004632 Acc: 13.3438
 |~~ train@6496  Loss: 0.005794 Acc: 13.1875
 |~~ train@6528  Loss: 0.005455 Acc: 13.3125
 |~~ train@6560  Loss: 0.005818 Acc: 13.1875
 |~~ train@6592  Loss: 0.005156 Acc: 13.3750
 |~~ train

 |~~ train@11744  Loss: 0.003264 Acc: 13.5938
 |~~ train@11776  Loss: 0.003405 Acc: 13.7188
 |~~ train@11808  Loss: 0.006094 Acc: 13.2188
 |~~ train@11840  Loss: 0.005414 Acc: 13.2812
 |~~ train@11872  Loss: 0.006507 Acc: 13.0312
 |~~ train@11904  Loss: 0.003906 Acc: 13.4688
 |~~ train@11936  Loss: 0.003760 Acc: 13.6250
 |~~ train@11968  Loss: 0.004768 Acc: 13.4688
 |~~ train@12000  Loss: 0.006853 Acc: 13.1250
 |~~ train@12032  Loss: 0.004853 Acc: 13.3125
 |~~ train@12064  Loss: 0.004350 Acc: 13.4375
 |~~ train@12096  Loss: 0.006297 Acc: 13.1250
 |~~ train@12128  Loss: 0.005383 Acc: 13.3125
 |~~ train@12160  Loss: 0.004308 Acc: 13.4375
 |~~ train@12192  Loss: 0.004401 Acc: 13.4688
 |~~ train@12224  Loss: 0.004732 Acc: 13.3125
 |~~ train@12256  Loss: 0.006270 Acc: 13.0625
 |~~ train@12288  Loss: 0.004068 Acc: 13.5000
 |~~ train@12320  Loss: 0.004985 Acc: 13.4062
 |~~ train@12352  Loss: 0.005243 Acc: 13.2812
 |~~ train@12384  Loss: 0.006762 Acc: 13.0000
 |~~ train@12416  Loss: 0.007597 A

 |~~ train@17504  Loss: 0.003625 Acc: 13.5938
 |~~ train@17536  Loss: 0.006033 Acc: 13.1875
 |~~ train@17568  Loss: 0.005913 Acc: 13.1250
 |~~ train@17600  Loss: 0.005135 Acc: 13.2188
 |~~ train@17632  Loss: 0.004478 Acc: 13.3750
 |~~ train@17664  Loss: 0.005227 Acc: 13.1875
 |~~ train@17696  Loss: 0.005005 Acc: 13.2500
 |~~ train@17728  Loss: 0.005557 Acc: 13.1875
 |~~ train@17760  Loss: 0.005741 Acc: 13.2188
 |~~ train@17792  Loss: 0.005785 Acc: 13.0625
 |~~ train@17824  Loss: 0.006844 Acc: 12.9688
 |~~ train@17856  Loss: 0.004209 Acc: 13.4375
 |~~ train@17888  Loss: 0.005732 Acc: 13.1875
 |~~ train@17920  Loss: 0.006086 Acc: 13.1562
 |~~ train@17952  Loss: 0.005344 Acc: 13.2188
 |~~ train@17984  Loss: 0.004232 Acc: 13.3750
 |~~ train@18016  Loss: 0.006395 Acc: 13.0625
 |~~ train@18048  Loss: 0.003490 Acc: 13.6250
 |~~ train@18080  Loss: 0.004586 Acc: 13.3750
 |~~ train@18112  Loss: 0.006838 Acc: 12.9062
 |~~ train@18144  Loss: 0.005031 Acc: 13.3438
 |~~ train@18176  Loss: 0.005109 A

 |~~ train@23232  Loss: 0.004303 Acc: 13.3438
 |~~ train@23264  Loss: 0.007118 Acc: 12.9375
 |~~ train@23296  Loss: 0.005299 Acc: 13.3125
 |~~ train@23328  Loss: 0.005061 Acc: 13.3125
 |~~ train@23360  Loss: 0.006165 Acc: 13.1562
 |~~ train@23392  Loss: 0.007901 Acc: 12.7188
 |~~ train@23424  Loss: 0.004321 Acc: 13.3750
 |~~ train@23456  Loss: 0.003998 Acc: 13.4375
 |~~ train@23488  Loss: 0.004748 Acc: 13.4375
 |~~ train@23520  Loss: 0.003899 Acc: 13.4375
 |~~ train@23552  Loss: 0.004106 Acc: 13.4375
 |~~ train@23584  Loss: 0.006152 Acc: 13.0938
 |~~ train@23616  Loss: 0.005382 Acc: 13.2500
 |~~ train@23648  Loss: 0.004524 Acc: 13.3750
 |~~ train@23680  Loss: 0.005527 Acc: 13.2500
 |~~ train@23712  Loss: 0.005247 Acc: 13.1562
 |~~ train@23744  Loss: 0.005100 Acc: 13.3125
 |~~ train@23776  Loss: 0.002736 Acc: 13.6875
 |~~ train@23808  Loss: 0.004065 Acc: 13.4688
 |~~ train@23840  Loss: 0.004184 Acc: 13.4062
 |~~ train@23872  Loss: 0.004951 Acc: 13.2188
 |~~ train@23904  Loss: 0.005312 A

...

 |~~ train@76928  Loss: 0.003976 Acc: 13.2500
 |~~ train@76960  Loss: 0.003072 Acc: 13.5938
 |~~ train@76992  Loss: 0.002909 Acc: 13.4688
 |~~ train@77024  Loss: 0.003176 Acc: 13.5312
 |~~ train@77056  Loss: 0.002945 Acc: 13.4062
 |~~ train@77088  Loss: 0.002939 Acc: 13.4375
 |~~ train@77120  Loss: 0.003808 Acc: 13.3750
 |~~ train@77152  Loss: 0.003194 Acc: 13.4688
 |~~ train@77184  Loss: 0.005675 Acc: 13.0312
 |~~ train@77216  Loss: 0.003993 Acc: 13.3438
 |~~ train@77248  Loss: 0.003969 Acc: 13.2500
 |~~ train@77280  Loss: 0.004344 Acc: 13.3750
 |~~ train@77312  Loss: 0.002885 Acc: 13.5938
 |~~ train@77344  Loss: 0.003095 Acc: 13.5000
 |~~ train@77376  Loss: 0.005946 Acc: 12.8750
 |~~ train@77408  Loss: 0.003325 Acc: 13.4688
 |~~ train@77440  Loss: 0.003768 Acc: 13.3438
 |~~ train@77472  Loss: 0.004384 Acc: 13.3438
 |~~ train@77504  Loss: 0.003122 Acc: 13.5312
 |~~ train@77536  Loss: 0.003760 Acc: 13.3438
 |~~ train@77568  Loss: 0.004058 Acc: 13.3438
 |~~ train@77600  Loss: 0.005022 A

 |~~ val@4480  Loss: 0.004387 Acc: 13.4375
 |~~ val@4512  Loss: 0.005036 Acc: 13.2812
 |~~ val@4544  Loss: 0.004491 Acc: 13.2500
 |~~ val@4576  Loss: 0.005766 Acc: 13.1562
 |~~ val@4608  Loss: 0.006424 Acc: 13.0938
 |~~ val@4640  Loss: 0.003990 Acc: 13.4062
 |~~ val@4672  Loss: 0.005163 Acc: 13.1875
 |~~ val@4704  Loss: 0.004572 Acc: 13.2500
 |~~ val@4736  Loss: 0.003207 Acc: 13.4688
 |~~ val@4768  Loss: 0.004813 Acc: 13.1562
 |~~ val@4800  Loss: 0.005930 Acc: 13.1250
 |~~ val@4832  Loss: 0.003437 Acc: 13.5938
 |~~ val@4864  Loss: 0.005021 Acc: 13.3125
 |~~ val@4896  Loss: 0.004429 Acc: 13.2812
 |~~ val@4928  Loss: 0.004691 Acc: 13.2812
 |~~ val@4960  Loss: 0.005293 Acc: 13.1250
 |~~ val@4992  Loss: 0.005572 Acc: 13.2500
 |~~ val@5024  Loss: 0.004434 Acc: 13.3750
 |~~ val@5056  Loss: 0.005387 Acc: 13.3125
 |~~ val@5088  Loss: 0.005749 Acc: 13.0312
 |~~ val@5120  Loss: 0.003524 Acc: 13.5938
 |~~ val@5152  Loss: 0.004595 Acc: 13.2500
 |~~ val@5184  Loss: 0.005085 Acc: 13.2812
 |~~ val@52

 |~~ val@10688  Loss: 0.007277 Acc: 12.8750
 |~~ val@10720  Loss: 0.004521 Acc: 13.4375
 |~~ val@10752  Loss: 0.004981 Acc: 13.3125
 |~~ val@10784  Loss: 0.005614 Acc: 13.2812
 |~~ val@10816  Loss: 0.004709 Acc: 13.3125
 |~~ val@10848  Loss: 0.005059 Acc: 13.2812
 |~~ val@10880  Loss: 0.004530 Acc: 13.3125
 |~~ val@10912  Loss: 0.005108 Acc: 13.2188
 |~~ val@10944  Loss: 0.003541 Acc: 13.4688
 |~~ val@10976  Loss: 0.004669 Acc: 13.1250
 |~~ val@11008  Loss: 0.004003 Acc: 13.3438
 |~~ val@11040  Loss: 0.005206 Acc: 13.2188
 |~~ val@11072  Loss: 0.005922 Acc: 13.3125
 |~~ val@11104  Loss: 0.003654 Acc: 13.4062
 |~~ val@11136  Loss: 0.003980 Acc: 13.4062
 |~~ val@11168  Loss: 0.006221 Acc: 13.0312
 |~~ val@11200  Loss: 0.005497 Acc: 13.1875
 |~~ val@11232  Loss: 0.005683 Acc: 13.0625
 |~~ val@11264  Loss: 0.005417 Acc: 13.0938
 |~~ val@11296  Loss: 0.004543 Acc: 13.2812
 |~~ val@11328  Loss: 0.003547 Acc: 13.5000
 |~~ val@11360  Loss: 0.006257 Acc: 12.9062
 |~~ val@11392  Loss: 0.005251 A

 |~~ val@16672  Loss: 0.004143 Acc: 13.4688
 |~~ val@16704  Loss: 0.004934 Acc: 13.2500
 |~~ val@16736  Loss: 0.005194 Acc: 13.2500
 |~~ val@16768  Loss: 0.004665 Acc: 13.1562
 |~~ val@16800  Loss: 0.005495 Acc: 13.3125
 |~~ val@16832  Loss: 0.004682 Acc: 13.2500
 |~~ val@16864  Loss: 0.004198 Acc: 13.3125
 |~~ val@16896  Loss: 0.004245 Acc: 13.5312
 |~~ val@16928  Loss: 0.004866 Acc: 13.2812
 |~~ val@16960  Loss: 0.004898 Acc: 13.3438
 |~~ val@16992  Loss: 0.005474 Acc: 13.1875
 |~~ val@17024  Loss: 0.003348 Acc: 13.5312
 |~~ val@17056  Loss: 0.003488 Acc: 13.3750
 |~~ val@17088  Loss: 0.005224 Acc: 13.3438
 |~~ val@17120  Loss: 0.006535 Acc: 12.9688
 |~~ val@17152  Loss: 0.002704 Acc: 13.5312
 |~~ val@17184  Loss: 0.003599 Acc: 13.4688
 |~~ val@17216  Loss: 0.006538 Acc: 12.9688
 |~~ val@17248  Loss: 0.004112 Acc: 13.4688
 |~~ val@17280  Loss: 0.006633 Acc: 12.9062
 |~~ val@17312  Loss: 0.005522 Acc: 13.0938
 |~~ val@17344  Loss: 0.003478 Acc: 13.4375
 |~~ val@17376  Loss: 0.003671 A

DataParallel(
  (module): ResNet(
    (conv1): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d (64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, ep

### Save model results to S3

aws s3 cp ResNet18PlusFlexibleFC_Epoch9.tar s3://bdh-xrayproj-modelparameters/

import boto3

s3 = boto3.client('s3')
s3.list_buckets()

S3 Commands: http://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html

Boto3 QuickStart: http://boto3.readthedocs.io/en/latest/guide/quickstart.html

Key Management: https://aws.amazon.com/blogs/security/a-safer-way-to-distribute-aws-credentials-to-ec2/

AWS IAM Rules: http://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html